In [ ]:
import numpy as np
import pylab as plt
import svgwrite
from skimage import io, filters, measure, feature, exposure
from scipy import ndimage, spatial
from scipy.ndimage import rotate
import os
import random
from IPython.display import SVG
%matplotlib inline

In [ ]:
ao_monkey = io.imread('TestMonkey/AO.png')
uv_monkey = io.imread('TestMonkey/UV.png')
normal_monkey = io.imread('TestMonkey/Normal.png')
original_monkey = io.imread('TestMonkey/Original.png')

fig=plt.figure(figsize=(20, 20))

fig.add_subplot(2, 2, 1)
plt.imshow(ao_monkey)
fig.add_subplot(2, 2, 2)
plt.imshow(uv_monkey)
fig.add_subplot(2, 2, 3)
plt.imshow(normal_monkey)
fig.add_subplot(2, 2, 4)
plt.imshow(original_monkey)
fig.tight_layout()

In [ ]:
edge_monkey = filters.sobel(ao_monkey[:,:,0])
coords = feature.corner_peaks(feature.corner_fast(edge_monkey), min_distance=5)
print(coords)
plt.figure()
plt.plot(coords[:,1],coords[:,0],"ro")
plt.imshow(edge_monkey)

In [ ]:
def center_angle(p1, p2):
    x_difference = p2[1]-p1[1]
    y_difference = p2[0]-p1[0]
    return np.rad2deg(np.arctan2(y_difference, x_difference))

In [ ]:
mean_x = np.mean(coords[1,:])
mean_y = np.mean(coords[0,:])
mean_point = [mean_y, mean_x]

start = coords[np.where(spatial.distance.cdist([[0, 0]], coords).max())]
angles = []
for i in coords:
    angles.append(center_angle(i, mean_point))

angles = np.asarray(angles)
coords = coords[angles.argsort()]
print(start, coords)
plt.figure()
plt.plot(coords[:,1],coords[:,0],)
plt.imshow(edge_monkey)

In [ ]:
dwg = svgwrite.Drawing('stroke.svg')

for i in range(0,10):
    genid = chr(i+48)
    gradline = dwg.linearGradient((0, 0), (1, 0), id=genid)
    gradline.add_stop_color(1, 'white', 0)
    gradline.add_stop_color(round(random.uniform(0.85, 0.95), 2), 'black')

    dwg.defs.add(gradline)
    
    start_x = random.randint(10, 100)
    start_y = random.randint(10, 100)
    
    dwg.add(dwg.line((start_x, start_y),
                     (start_x + random.randint(25, 100), start_y + random.randint(25, 100)),
                     stroke="url(#" + genid + ")",
                     stroke_linecap="round"))

display(SVG(dwg.tostring().replace('><','>\n<')))

dwg.save()